# Solution tRAT 3

---
---

**Author:** Dr Giordano Scarciotti (g.scarciotti@imperial.ac.uk) - Imperial College London

**Module:** ELEC70066 - Applied Advanced Optimisation

**Version:** 1.1.1 - 29/01/2025

---
---

# Exercise

Consider a three-state boolean linear programme

$$
\begin{array}{ll}
\displaystyle \min_{x} &  c^\top x \\
\text{s.t. } & Ax \preccurlyeq b,\\
& x_i\in \{0, 0.5, 1\}, \qquad i = 1, \dots, n.
\end{array}\tag{1}
$$

where the variable $x$ is constrained to have components equal to zero, half or one. You can think of $x_i$ as a job we can accept (1), subcontract (0.5) or decline (0), and $−c_i$ as the (positive) revenue we generate if we accept or subcontract job $i$. We can think of $Ax \preccurlyeq b$ as a set of limits on $m$ resources. $A_{ij}$, which is positive, is the amount of resource $i$ consumed if we accept job $j$; If the job is subcontracted we have half the revenues, but we also half the amount of resources spent. $b_i$, which is positive, is the amount of resource $i$ available.



For your convenience, the relaxation of the problem (i.e. $0 \le x\le 1$) is already solved in the code below.

In [ ]:
import cvxpy as cp
import numpy as np

n = 100
m = 300
np.random.seed(1)
A = np.random.random((m, n)) # positve
b = np.dot(A,np.ones(n)/2) # postive
c = -np.random.random(n) # negative, so -c is positive

In [ ]:
# Define and solve the CVXPY problem.
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize(c.T@x),[A @ x <= b, x >= 0, x<= 1])
prob.solve()

# Print result.
L=prob.value
print("\nThe optimal value is", prob.value)
#print("A solution x is", x.value)


The optimal value is -34.589711316657315


1.   Explain how you would develop a threshold rounding for this problem.
2.   Carry out threshold rounding. For each value of of the threshold(s), note the objective value $c^\top \hat{x}$ and the maximum constraint violation $\max_i(A\hat{x} − b)_i$.
3.   Find the minimum feasible objective value and note it as the upper bound $U$. Compute the relative suboptimality percentage as $\frac{|U-L|}{|L|} \times 100$.



# Solution

1.   The relaxed solution $x_r$ can be used to guess a point $\hat x$ by rounding its entries based on two thresholds $t_b \in [0,1]$ and $t^t \in [0,1]$:
$$
\hat x_i = \left\{ \begin{array}{ll} 1 & x_r \ge t^t \\
0.5 & t_b \le x_r < t^t \\
 0 & x_r \le t_b \end{array} \right.
$$
for $i = 1,\dots, n$. If $\hat x$ is feasible for problem $(1)$, i.e., if $A\hat x \preccurlyeq b$, then it can be considered a guess at a good point for the Boolean problem. Its objective value, $U = c^\top \hat{x}$, is an upper bound on $p_B^*$. If $U$ and $L$ are close, then $\hat{x}$ is nearly optimal; specifically, $\hat{x}$ cannot be more than $(U − L)$-suboptimal for problem $(1)$.

2.   Threshold rounding is carried out in the following code.

In [ ]:
import matplotlib.pyplot as plt # Library to generate the plots

In [ ]:
# Generation of 100 values of tt and tb uniformly spaced over [0,1]^2
num_t=100
tt=np.linspace(0, 1, num=num_t)
tb=np.linspace(0, 1, num=num_t)

In [ ]:
# Implementation of the relaxation
obj = np.zeros((num_t,num_t)) # Initialisation of 2d-array of (100,100) objective values
maxviol = np.zeros((num_t,num_t)) # Initialisation of 2d-array of (100,100) maximum violations
for i in range(num_t): # for each threshold
  for j in range(num_t):
    hat_x = np.zeros(n) # we implement the rule
    for k in range(n):
        if x.value[k]>=tt[i]:
          hat_x[k] = 1
        elif x.value[k]>=tb[j] and x.value[k]<tt[i]:
          hat_x[k] = 0.5
        else:
          hat_x[k] = 0
    obj[i][j] = c.T@hat_x # and compute the objective value
    maxviol[i][j] = max(A @ hat_x - b) # and constraint

3.   The upper bound for $(1)$ is given by

In [ ]:
U=min(obj[np.where(maxviol <= 0)])
print("The upper bound is ",U)

The upper bound is  -33.99926712688335


In [ ]:
print("The relative suboptimality percentage is",round(abs((U-L)/L)*100,2),"%")

The relative suboptimality percentage is 1.71 %


## Addendum (not required)

Here we compute a feasible $\hat x$ for $(1)$ which gives objective value equal to the upper bound.

In [ ]:
# We find the indexes of tt and tb corresponding to U
np.where(obj==U)

(array([56, 57, 58, 59, 60, 61, 62]), array([36, 36, 36, 36, 36, 36, 36]))

Any of the indeces pairs is equivalent. We pick $(56,36)$ and compute $\hat x$

In [ ]:
i = 56
j = 36
hat_x = np.zeros(n) # we implement the rule
for k in range(n):
  if x.value[k]>=tt[i]:
    hat_x[k] = 1
  elif x.value[k]>=tb[j] and x.value[k]<tt[i]:
    hat_x[k] = 0.5
  else:
    hat_x[k] = 0

The obtained $\hat{x}$ is given below.

In [ ]:
hat_x

array([0. , 1. , 1. , 0. , 0. , 1. , 1. , 1. , 1. , 1. , 1. , 0. , 1. ,
       0. , 0. , 0. , 1. , 1. , 0. , 0. , 0.5, 1. , 0. , 1. , 0. , 0. ,
       0. , 0. , 0. , 0.5, 0. , 1. , 0. , 1. , 1. , 0. , 0. , 1. , 1. ,
       0. , 1. , 0. , 0. , 1. , 1. , 1. , 1. , 0. , 0. , 1. , 0. , 0. ,
       1. , 1. , 0. , 0. , 0. , 1. , 1. , 1. , 1. , 0. , 0. , 0. , 1. ,
       0. , 0. , 1. , 1. , 0. , 0. , 0. , 0.5, 0. , 1. , 0. , 0. , 0. ,
       1. , 1. , 0. , 0. , 0. , 0. , 1. , 0. , 1. , 0. , 0. , 0. , 1. ,
       1. , 0. , 1. , 0. , 1. , 0. , 1. , 0. , 0. ])

And the rounding threshold is

In [ ]:
print("(",tt[56],",",tb[36],")")

( 0.5656565656565657 , 0.36363636363636365 )


We can also look for all equivalent thresholds.

In [ ]:
tt[np.where(obj==U)[0]]

array([0.56565657, 0.57575758, 0.58585859, 0.5959596 , 0.60606061,
       0.61616162, 0.62626263])

This shows that $t^t$ can be any value between $0.566$ and $0.626$, whereas $t_b$ must be equal to $0.364$. It is interesting to note that $t^t=0.666$ and $t_b=0.333$ is not an optimal solution.